In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# EXERCISE 1 SOLUTION
# Define a kernel that prepares the walker qubits in an
# equal superposition of the states |2> = |0010> and |3> = |0011>.

@cudaq.kernel
def initial_position(qubits : cudaq.qvector):
    """ Apply gates to the qubits to prepare the GHZ state
    Parameters
        qubits: cudaq.qvector
        qubits for the walker
    """
    # Edit the code below this line
    x(qubits[2])
    h(qubits[3])
    
    # Edit the code above this line

In [ ]:
# EXERCISE 2 SOLUTION
# Complete the step operation
# Edit the code to apply the DEC operation to the walker qubits
# when the coin qubit is in the |0> state

# Set the number of qubits
num_qubits = 4

@cudaq.kernel
def DTQW_one_step(num_qubits: int):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qubit()

    # Initial walker state 1/(sqrt{2}) ( |2>+|3>)
    initial_position(walker_qubits)

    # Initial coin state
    h(coin_qubit) #CHANGE_ME

    # One quantum walk step
    # Coin operation
    h(coin_qubit) #CHANGE_ME

    # Walker's position change

    ## Shifting right

    ## Shift right when the coin is |1>
    INC.ctrl(coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3])

    ## Shifting left
    ## Shift left when the coin is |0>
    ## EDIT CODE BELOW THIS LINE
       
    x(coin_qubit)
    DEC.ctrl(coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3])
    x(coin_qubit)

    ## EDIT CODE ABOVE THIS LINE

    # Measure the state of the walker qubits
    mz(walker_qubits)

# Visualize the kernel for the quantum walk
#print(cudaq.draw(DTQW_one_step, num_qubits))

# Sample the kernel for the quantum walk
result = cudaq.sample(DTQW_one_step, num_qubits, shots_count=1000)
print(result)

plot_results(result, num_qubits)

In [ ]:
# EXERCISE 3 SOLUTION
# Kernel to change a coin from 1 to 0 if the walker is the state |1111>
@cudaq.kernel
def no_INC_at_right_endpoint(walker_qubits : cudaq.qvector, coin_qubit : cudaq.qubit, right_endpoint : cudaq.qubit):
    
    ## EDIT CODE BELOW THIS POINT##
    
    # Test if the coin is in |1> and the walker state is |1111>,  if so, change the end_point qubit to 1. 
    x.ctrl([coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3]], right_endpoint)
    
    # Flip the state of the coin if the endpoint is triggered. 
    # Hint, this can be done in one line.
    x.ctrl(right_endpoint, coin_qubit)

    ## EDIT CODE ABOVE THIS POINT##

    

# Kernel to change a coin from 0 to 1 if the walker is in the state |0000>
@cudaq.kernel
def no_DEC_at_left_endpoint(walker_qubits : cudaq.qvector, coin_qubit : cudaq.qubit, left_endpoint : cudaq.qubit):
    # Bit Flip the walker and coin qubits
    x(coin_qubit)
    x(walker_qubits)
    
    # Trigger the left_endpoint qubit if the walker and the coin are now in the states |1> and |1111> respectively 
    x.ctrl([coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3]], left_endpoint)
    
    # Undo the bit flip on the walker and coin qubits
    x(walker_qubits)
    x(coin_qubit)
    
    # Flip the coin fron |0> to |1> if the endpoint qubit is triggered
    x.ctrl(left_endpoint,coin_qubit)
   
# Kernel to reset the coin and endpoint qubit
@cudaq.kernel()
def reset_coin_and_endpoint(coin_qubit : cudaq.qubit, endpoint: cudaq.qubit):
    # change the coin qubit back if it was flipped to prevent transitions
    # between |0000> and |1111>
    x.ctrl(endpoint, coin_qubit)

    # reset the endpoint qubit to |0>
    reset(endpoint)